In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import pickle
from tensorflow.python.framework import ops
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.metrics import f1_score, accuracy_score, classification_report
from sklearn.utils.validation import check_X_y, check_array, check_is_fitted
from sklearn.model_selection import GridSearchCV

In [2]:
class RBFKernelSVM(BaseEstimator, ClassifierMixin) :
    """SVM with RBF kernel Classifier implemented with Tensorflow and compatible to SkLearn Estimator"""
    
    def __init__(self, gamma, C=1.0, verbose=False) :
        self.gamma = gamma
        self.C = C
        self.verbose = verbose

    def fit(self, X, y) : 
        X, y = check_X_y(X, y)
        
        self.xData_ = X
        self.yTarget_ = y
        
        # Step 1: Reinitialize the Tensorflow graph
        ops.reset_default_graph()
        self.sess_ = tf.Session()
        
        # Step 2: Create the Placeholders
        featuresSize = np.shape(X)[1]
        trainingDataSize = np.shape(X)[0]
        self.xInput_ = tf.placeholder(shape=[None, featuresSize], dtype=np.float32)
        self.yInput_ = tf.placeholder(shape=[None, 1], dtype=np.int32)
        self.prediction_grid_ = tf.placeholder(shape=[None, featuresSize], dtype=np.float32)
        self.xInputIndexes_ = tf.placeholder(shape=[None], dtype=np.int32)
        
        # Step 3: Initialize the model parameters
        b = tf.Variable(tf.random_normal(shape=[trainingDataSize, 1], dtype=np.float32))
        bEpoch = tf.gather(b, self.xInputIndexes_)
        
        # Step 4: Compute Training Kernel
        gammVal = tf.negative(tf.constant(self.gamma))
        dist = tf.reduce_sum(tf.square(self.xInput_), 1)
        dist = tf.reshape(dist, [-1, 1])
        sq_dists = tf.add(tf.subtract(dist, tf.multiply(2.0, tf.matmul(self.xInput_, tf.transpose(self.xInput_)))), tf.transpose(dist))
        my_kernel = tf.exp(tf.multiply(gammVal, tf.abs(sq_dists)))
        
        # Step 5: Compute the Loss
        first_term = tf.reduce_sum(bEpoch)
        b_vec_cross = tf.matmul(bEpoch, tf.transpose(bEpoch))
        y_target_cross = tf.matmul(tf.cast(self.yInput_, np.float32), tf.transpose(tf.cast(self.yInput_, np.float32)))
        second_term = tf.multiply(0.5, tf.reduce_sum(tf.multiply(tf.multiply(y_target_cross, my_kernel), b_vec_cross)))
        loss = tf.subtract(second_term, first_term)
        
        # Step 6: Compute Prediction Kernel
        rA = tf.reshape(tf.reduce_sum(tf.square(self.xInput_), 1), [-1, 1])
        rB = tf.reshape(tf.reduce_sum(tf.square(self.prediction_grid_), 1), [-1, 1])
        pred_sq_dist = tf.add(tf.subtract(rA, tf.multiply(2.0, tf.matmul(self.xInput_, tf.transpose(self.prediction_grid_)))), tf.transpose(rB))
        pred_kernel = tf.exp(tf.multiply(gammVal, tf.abs(pred_sq_dist)))
        
        # Step 7: Compute the Prediction
        prediction_output = tf.matmul(tf.multiply(tf.transpose(tf.cast(self.yInput_, np.float32)), tf.transpose(bEpoch)), tf.transpose(pred_kernel))
        self.prediction_ = tf.squeeze(tf.sign(prediction_output - tf.reduce_mean(prediction_output)))
        
        # Step 8: Optimizer
        optimizer = tf.train.GradientDescentOptimizer(0.001)
        train_step = optimizer.minimize(loss)
        
        # Step 9: Update the B variable
        tf.scatter_update(b, self.xInputIndexes_, bEpoch)
        
        # Step 10: Initialize the session
        init = tf.global_variables_initializer()
        self.sess_.run(init)
        
        # Step 10: Train the model
        epochs = 1000
        batchSize = 500
        np.random.seed(seed=2)
        for epoch in range(epochs) :
            stepCounter = 0
            indexes = np.arange(len(y))
            np.random.shuffle(indexes)
            currIndex = batchSize
            while currIndex <= len(y) :
                stepCounter = stepCounter + 1
                currIndexes = indexes[currIndex-batchSize:currIndex]
                batchX = X[currIndexes]
                batchY = np.reshape(y[currIndexes], newshape=[-1, 1])
                _, lossVal, predictionValue = self.sess_.run([train_step, loss, self.prediction_], feed_dict={self.xInput_:batchX, self.yInput_:batchY, self.prediction_grid_:batchX, self.xInputIndexes_:currIndexes})
                f1Score = f1_score(batchY, predictionValue, average='weighted')
                accuracy = accuracy_score(batchY, predictionValue)
                if self.verbose :
                    print("Epoch {0}, Step {1}    Loss: {2}    F1_Score: {3}   Accuracy: {4}".format(epoch, stepCounter, lossVal, f1Score, accuracy))
                currIndex = currIndex + batchSize
        
        if self.verbose :
            print("End of the training")
            
        return self
        
    def predict(self, X) :
        X = check_array(X)
        check_is_fitted(self, ['xData_'])
        output = self.sess_.run(self.prediction_, feed_dict={self.xInput_:X, self.yInput_:np.reshape(self.yTarget_, newshape=[-1, 1]), self.prediction_grid_:self.xData_, self.xInputIndexes_:np.arange(len(self.yTarget_))})  
        return output
    
    def score(self, X, y) :
        prediction = self.predict(X)
        accuracy = accuracy_score(y, prediction)
        if self.verbose :
            print("Accuracy Score is: {0}".format(accuracy))
        return f1_score(y, prediction, average='weighted')
    
    def classificationReport(self, X, y) :
        prediction = self.predict(X)
        return classification_report(y, prediction, output_dict=True)
        

In [3]:
# Build a Training and Test sets
dfTrain = pd.read_pickle('../data/dfTrain.pickle')
dfTest = pd.read_pickle('../data/dfTest.pickle')

trainData = dfTrain[['featureOne', 'featureTwo', 'featureThree', 'featureFour']].values
trainLabel = dfTrain['label'].values
testData = dfTest[['featureOne', 'featureTwo', 'featureThree', 'featureFour']].values
testLabel = dfTest['label'].values

print("The shape of the training data is {}".format(trainData.shape))
print("The shape of the training labels is {}".format(trainLabel.shape))
print("The shape of the test data is {}".format(testData.shape))
print("The shape of the test label is {}".format(testLabel.shape))

The shape of the training data is (3089, 4)
The shape of the training labels is (3089,)
The shape of the test data is (4000, 4)
The shape of the test label is (4000,)


In [4]:
# Perform coarse GridSearch to search for the optimal region of the parameters C and gamma
# Search C on [2^-3, ..., 2^3] and gamma on [2^-3, ..., 2^3]
cRange = np.arange(-3, 3, dtype=np.float32)
gRange = np.arange(-3, 3, dtype=np.float32)
paramsDict = {'gamma':np.power(2.0, gRange), 'C':np.power(2.0, cRange)}
svm = RBFKernelSVM(1.0)
grid = GridSearchCV(estimator=svm, param_grid=paramsDict, cv=5)
grid.fit(trainData, trainLabel)
print(grid.best_params_)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: F-score is ill-defined an

{'C': 4.0, 'gamma': 0.5}


In [5]:
# Perform finer GridSearch on the Neighborhood of the optimal value (4.0, 0.5)
finerGridOnC = np.arange(1.90, 2.15, 0.05, dtype=np.float32)
finerGridOnGamma = np.arange(-1.15, -0.85, 0.05, dtype=np.float32)
paramsDict = {'gamma':np.power(2.0, finerGridOnGamma), 'C':np.power(2.0, finerGridOnC)}
svm = RBFKernelSVM(1.0)
grid = GridSearchCV(estimator=svm, param_grid=paramsDict, cv=5)
grid.fit(trainData, trainLabel)
print(grid.best_params_)

/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


{'C': 3.8637455, 'gamma': 0.5176324}


In [6]:
# Train SVM model with the Optimal Parameter value
model = RBFKernelSVM(gamma=0.5176324, C=3.8637455, verbose=True)
model.fit(trainData, trainLabel)

Epoch 0, Step 1    Loss: 176.220703125    F1_Score: 0.5860228667398747   Accuracy: 0.58
Epoch 0, Step 2    Loss: 175.90689086914062    F1_Score: 0.5016696403906465   Accuracy: 0.488
Epoch 0, Step 3    Loss: 286.991455078125    F1_Score: 0.5768866995073892   Accuracy: 0.564
Epoch 0, Step 4    Loss: 111.17066192626953    F1_Score: 0.605358844523469   Accuracy: 0.602
Epoch 0, Step 5    Loss: 293.0943908691406    F1_Score: 0.48409550798914674   Accuracy: 0.478
Epoch 0, Step 6    Loss: 245.1942138671875    F1_Score: 0.4646390145365769   Accuracy: 0.454
Epoch 1, Step 1    Loss: 191.61221313476562    F1_Score: 0.5028955745341614   Accuracy: 0.492
Epoch 1, Step 2    Loss: 107.10726928710938    F1_Score: 0.3102230832829448   Accuracy: 0.298
Epoch 1, Step 3    Loss: 372.13934326171875    F1_Score: 0.5425599897178843   Accuracy: 0.536
Epoch 1, Step 4    Loss: 236.41848754882812    F1_Score: 0.5321249597423511   Accuracy: 0.524
Epoch 1, Step 5    Loss: 152.08460998535156    F1_Score: 0.54606785942

Epoch 18, Step 5    Loss: 199.5609130859375    F1_Score: 0.6033415929203539   Accuracy: 0.594
Epoch 18, Step 6    Loss: 198.49667358398438    F1_Score: 0.27847069917822975   Accuracy: 0.262
Epoch 19, Step 1    Loss: 286.9030456542969    F1_Score: 0.5932523665847125   Accuracy: 0.578
Epoch 19, Step 2    Loss: 133.50987243652344    F1_Score: 0.7736906186885826   Accuracy: 0.77
Epoch 19, Step 3    Loss: 140.10794067382812    F1_Score: 0.47007907979870595   Accuracy: 0.456
Epoch 19, Step 4    Loss: 230.33592224121094    F1_Score: 0.7098938909468249   Accuracy: 0.706
Epoch 19, Step 5    Loss: 170.2515869140625    F1_Score: 0.6080808762721304   Accuracy: 0.6
Epoch 19, Step 6    Loss: 550.0498657226562    F1_Score: 0.4856128881395149   Accuracy: 0.478
Epoch 20, Step 1    Loss: 367.84710693359375    F1_Score: 0.7138219656283566   Accuracy: 0.708
Epoch 20, Step 2    Loss: 277.96160888671875    F1_Score: 0.5176138828633405   Accuracy: 0.51
Epoch 20, Step 3    Loss: 98.27181243896484    F1_Score:

Epoch 37, Step 6    Loss: 203.40533447265625    F1_Score: 0.7917889683582283   Accuracy: 0.788
Epoch 38, Step 1    Loss: 214.64593505859375    F1_Score: 0.36511458952513043   Accuracy: 0.36
Epoch 38, Step 2    Loss: 67.96709442138672    F1_Score: 0.6510378660461744   Accuracy: 0.642
Epoch 38, Step 3    Loss: 155.89859008789062    F1_Score: 0.6388056171470805   Accuracy: 0.63
Epoch 38, Step 4    Loss: 203.7259063720703    F1_Score: 0.5461496180248677   Accuracy: 0.538
Epoch 38, Step 5    Loss: 168.43687438964844    F1_Score: 0.40395805965262593   Accuracy: 0.39
Epoch 38, Step 6    Loss: 111.37728881835938    F1_Score: 0.5306199572779982   Accuracy: 0.518
Epoch 39, Step 1    Loss: 187.35269165039062    F1_Score: 0.5086341199429312   Accuracy: 0.5
Epoch 39, Step 2    Loss: 185.4493408203125    F1_Score: 0.4363029474917041   Accuracy: 0.42
Epoch 39, Step 3    Loss: 194.2976531982422    F1_Score: 0.6562285843740264   Accuracy: 0.65
Epoch 39, Step 4    Loss: 166.46432495117188    F1_Score: 0

Epoch 55, Step 2    Loss: 198.19964599609375    F1_Score: 0.7891494682715905   Accuracy: 0.784
Epoch 55, Step 3    Loss: 189.6259765625    F1_Score: 0.5807619594258667   Accuracy: 0.574
Epoch 55, Step 4    Loss: 271.5231628417969    F1_Score: 0.45884463510944284   Accuracy: 0.448
Epoch 55, Step 5    Loss: 204.19654846191406    F1_Score: 0.4168871055549954   Accuracy: 0.406
Epoch 55, Step 6    Loss: 199.49441528320312    F1_Score: 0.565157289090828   Accuracy: 0.554
Epoch 56, Step 1    Loss: 170.79229736328125    F1_Score: 0.7993779220779222   Accuracy: 0.796
Epoch 56, Step 2    Loss: 271.9668884277344    F1_Score: 0.6422397417381701   Accuracy: 0.634
Epoch 56, Step 3    Loss: 126.36061096191406    F1_Score: 0.6973850893407271   Accuracy: 0.694
Epoch 56, Step 4    Loss: 340.4276428222656    F1_Score: 0.4646973210715713   Accuracy: 0.454
Epoch 56, Step 5    Loss: 163.0175323486328    F1_Score: 0.6724684610643106   Accuracy: 0.664
Epoch 56, Step 6    Loss: 311.8042907714844    F1_Score: 0

Epoch 73, Step 4    Loss: 172.94285583496094    F1_Score: 0.33188835316854853   Accuracy: 0.318
Epoch 73, Step 5    Loss: 100.47667694091797    F1_Score: 0.5080887858610502   Accuracy: 0.498
Epoch 73, Step 6    Loss: 160.87791442871094    F1_Score: 0.7117855316550696   Accuracy: 0.706
Epoch 74, Step 1    Loss: 139.37442016601562    F1_Score: 0.46935360291882033   Accuracy: 0.468
Epoch 74, Step 2    Loss: 279.6302490234375    F1_Score: 0.3500141360140376   Accuracy: 0.336
Epoch 74, Step 3    Loss: 125.43951416015625    F1_Score: 0.6320877239199156   Accuracy: 0.62
Epoch 74, Step 4    Loss: 127.21358489990234    F1_Score: 0.48213240958936776   Accuracy: 0.474
Epoch 74, Step 5    Loss: 395.48199462890625    F1_Score: 0.7248695652173912   Accuracy: 0.72
Epoch 74, Step 6    Loss: 324.53302001953125    F1_Score: 0.650894636109339   Accuracy: 0.64
Epoch 75, Step 1    Loss: 165.55596923828125    F1_Score: 0.5945065216329323   Accuracy: 0.584
Epoch 75, Step 2    Loss: 145.10128784179688    F1_S

...

Epoch 937, Step 3    Loss: -16.89592742919922    F1_Score: 0.6813861453080542   Accuracy: 0.678
Epoch 937, Step 4    Loss: -19.88641357421875    F1_Score: 0.6919552901546471   Accuracy: 0.68
Epoch 937, Step 5    Loss: -6.111736297607422    F1_Score: 0.5672034161490683   Accuracy: 0.556
Epoch 937, Step 6    Loss: 0.7835159301757812    F1_Score: 0.8140057061460121   Accuracy: 0.81
Epoch 938, Step 1    Loss: -7.60723876953125    F1_Score: 0.7510028985507247   Accuracy: 0.748
Epoch 938, Step 2    Loss: 17.34689712524414    F1_Score: 0.6258531502889001   Accuracy: 0.618
Epoch 938, Step 3    Loss: -17.01087188720703    F1_Score: 0.7544551724137932   Accuracy: 0.748
Epoch 938, Step 4    Loss: -13.6090087890625    F1_Score: 0.6950899843505479   Accuracy: 0.688
Epoch 938, Step 5    Loss: -5.1505279541015625    F1_Score: 0.6450328638497652   Accuracy: 0.636
Epoch 938, Step 6    Loss: 57.937191009521484    F1_Score: 0.4987275059707711   Accuracy: 0.486
Epoch 939, Step 1    Loss: -4.41062545776367

Epoch 955, Step 1    Loss: -21.556243896484375    F1_Score: 0.6178073770491803   Accuracy: 0.616
Epoch 955, Step 2    Loss: -3.9913711547851562    F1_Score: 0.8274064516129032   Accuracy: 0.824
Epoch 955, Step 3    Loss: -1.6686210632324219    F1_Score: 0.7098738783463443   Accuracy: 0.706
Epoch 955, Step 4    Loss: 36.98252868652344    F1_Score: 0.568774020674312   Accuracy: 0.554
Epoch 955, Step 5    Loss: 18.761367797851562    F1_Score: 0.4078168341294679   Accuracy: 0.398
Epoch 955, Step 6    Loss: -36.562808990478516    F1_Score: 0.8570912630810414   Accuracy: 0.854
Epoch 956, Step 1    Loss: -23.08526611328125    F1_Score: 0.5466543422580497   Accuracy: 0.53
Epoch 956, Step 2    Loss: -18.689926147460938    F1_Score: 0.7258514608921554   Accuracy: 0.72
Epoch 956, Step 3    Loss: -25.206100463867188    F1_Score: 0.6593331180156321   Accuracy: 0.656
Epoch 956, Step 4    Loss: -20.975914001464844    F1_Score: 0.6982949575057124   Accuracy: 0.694
Epoch 956, Step 5    Loss: 15.3174781

Epoch 972, Step 5    Loss: 12.914115905761719    F1_Score: 0.6025237172493528   Accuracy: 0.592
Epoch 972, Step 6    Loss: 22.31873321533203    F1_Score: 0.689860365198711   Accuracy: 0.68
Epoch 973, Step 1    Loss: -29.85657501220703    F1_Score: 0.7401093797837541   Accuracy: 0.736
Epoch 973, Step 2    Loss: 52.36947250366211    F1_Score: 0.358796992481203   Accuracy: 0.35
Epoch 973, Step 3    Loss: 14.483505249023438    F1_Score: 0.7225389068367799   Accuracy: 0.716
Epoch 973, Step 4    Loss: -24.047061920166016    F1_Score: 0.6933508483853311   Accuracy: 0.688
Epoch 973, Step 5    Loss: 8.928108215332031    F1_Score: 0.6995254220074394   Accuracy: 0.686
Epoch 973, Step 6    Loss: 7.671012878417969    F1_Score: 0.7682634747879388   Accuracy: 0.766
Epoch 974, Step 1    Loss: 10.006195068359375    F1_Score: 0.7688081494057725   Accuracy: 0.764
Epoch 974, Step 2    Loss: -2.015838623046875    F1_Score: 0.6465947187702386   Accuracy: 0.638
Epoch 974, Step 3    Loss: -32.29963302612305  

Epoch 990, Step 2    Loss: -21.668807983398438    F1_Score: 0.7433024236037934   Accuracy: 0.736
Epoch 990, Step 3    Loss: 3.115314483642578    F1_Score: 0.6104770879393038   Accuracy: 0.602
Epoch 990, Step 4    Loss: -35.400264739990234    F1_Score: 0.8383331061347742   Accuracy: 0.834
Epoch 990, Step 5    Loss: -16.375431060791016    F1_Score: 0.5659152709359605   Accuracy: 0.556
Epoch 990, Step 6    Loss: -8.604835510253906    F1_Score: 0.7032151260013595   Accuracy: 0.698
Epoch 991, Step 1    Loss: -33.10128402709961    F1_Score: 0.8776376254969341   Accuracy: 0.876
Epoch 991, Step 2    Loss: -11.333709716796875    F1_Score: 0.6743554586464943   Accuracy: 0.666
Epoch 991, Step 3    Loss: 15.184059143066406    F1_Score: 0.5434140869377628   Accuracy: 0.53
Epoch 991, Step 4    Loss: -32.91141128540039    F1_Score: 0.553003723419374   Accuracy: 0.542
Epoch 991, Step 5    Loss: 31.59329605102539    F1_Score: 0.6245667785576239   Accuracy: 0.618
Epoch 991, Step 6    Loss: -22.963653564

RBFKernelSVM(C=3.8637455, gamma=0.5176324, verbose=True)

In [7]:
modelReportOnTest = model.classificationReport(testData, testLabel)
print(modelReportOnTest)
with open('../data/svmWithRBFKernel.pickle', 'wb') as fp :
    pickle.dump(modelReportOnTest, fp)
print('Report Saved')

{'-1': {'precision': 0.9646941819990055, 'recall': 0.97, 'f1-score': 0.9673398155073547, 'support': 2000}, '1': {'precision': 0.9698340874811463, 'recall': 0.9645, 'f1-score': 0.9671596891451492, 'support': 2000}, 'accuracy': 0.96725, 'macro avg': {'precision': 0.9672641347400759, 'recall': 0.9672499999999999, 'f1-score': 0.967249752326252, 'support': 4000}, 'weighted avg': {'precision': 0.9672641347400759, 'recall': 0.96725, 'f1-score': 0.967249752326252, 'support': 4000}}
Report Saved
